In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

In [3]:
from pynq import Overlay
overlay = Overlay('./bitstream/DEC_rLUT24bits.bit')

In [4]:
overlay.ip_dict

{'DEC_rLUT24bits_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xaf80f6d0>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'DEC_rLUT24bits_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S00_AXI',
  'parameters': {'C_S00_AXI_ADDR_WIDTH': '4',
   'C_S00_AXI_BASEADDR': '0x43C00000',
   'C_S00_AXI_DATA_WIDTH': '32',
   'C_S00_AXI_HIGHADDR': '0x43C0FFFF',
   'Component_Name': 'DEC_rLUT24bits_DEC_rLUT24bits_0_0',
   'EDK_IPTYPE': 'PERIPHERAL'},
  'phys_addr': 1136656384,
  'registers': {},
  'state': None,
  'type': 'xilinx.com:user:DEC_rLUT24bits:1.0'}}

In [5]:
from pynq import MMIO
ip = MMIO(0x43C00000, 0x10000)

In [6]:
ip.device

### 將 r_LUT  的location輸出值轉為2補數:

In [7]:
def twos_complement_32bit(x):
    """將 32-bit 補數整數轉為有號值"""
    if x >= (1 << 31):
        return x - (1 << 32)
    else:
        return x

#### Test:

In [8]:
remainder_1 = 9908

ip.write_mm(0, remainder_1)

location_1 = ip.read_mm(4)
location_2 = ip.read_mm(8)


print(twos_complement_32bit(location_1))
print(twos_complement_32bit(location_2))

-19
30


### 開始則中查表驗證

## 折衷查表:
### 用 Single Error Correction 解 Double Error Correction (A 取Double_AWE的 A)

#### 1.取 A的bits $\approx$ N的bits $\Rightarrow$ A = 18613  (30bits), N的範圍: 0 ~ 1073741823 (30bits)

### Define function
* (Not including 2's complement range

In [9]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return unique_list


def get_sign(number):
    return 1 if number > 0 else -1 if number < 0 else 0


print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


In [10]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data1 = (1 << 24) - 21    ## 
data2 = (1 << 24) - 19
data3 = (1 << 24) - 15
AN = 13837
input1 = data1*AN
input2 = data2*AN
input3 = data3*AN
bits1 = bits_needed(input1)
bits2 = bits_needed(input2)
bits3 = bits_needed(input3)
print("inputs1的位元數:", bits1)
print("inputs2的位元數:", bits2)
print("inputs3的位元數:", bits3)

inputs1的位元數: 38
inputs2的位元數: 38
inputs3的位元數: 38


### 產生1-bit AWE (Arithmetic-Weight Error)
* 並產生 Delta Q 的 list

In [20]:
AWE1_1=[]
e1_1=0     ## 錯誤數量
e1_data1=[]
h = [];   #位置
for i in range(0, bits1):
    e1_data1.insert(e1_1 , input1 + 2**i)
    AWE1_1.insert(e1_1, 2**i)
    e1_1 = e1_1 + 1
    e1_data1.insert(e1_1 , input1 - 2**i)
    AWE1_1.insert(e1_1, -2**i)
    e1_1 = e1_1 + 1
    h.insert(i,i+1)

AWE1_2=[]
e1_2=0     ## 錯誤數量
e1_data2=[]
for i in range(0, bits2):
    e1_data2.insert(e1_2 , input2 + 2**i)
    AWE1_2.insert(e1_2, 2**i)
    e1_2 = e1_2 + 1
    e1_data2.insert(e1_2 , input2 - 2**i)
    AWE1_2.insert(e1_2, -2**i)
    e1_2 = e1_2 + 1
    h.insert(i,i+1)
    
AWE1_3=[]
e1_3=0     ## 錯誤數量
e1_data3=[]
for i in range(0, bits3):
    e1_data3.insert(e1_3 , input3 + 2**i)
    AWE1_3.insert(e1_3, 2**i)
    e1_3 = e1_3 + 1
    e1_data3.insert(e1_3 , input3 - 2**i)
    AWE1_3.insert(e1_3, -2**i)
    e1_3 = e1_3 + 1
    h.insert(i,i+1)

h_awe = [val for num in h for val in (num, -num)]
print("產生1-bit AWE:", AWE1_1)
print("不同輸入資料的AWE是否相等:",AWE1_1 == AWE1_2 == AWE1_3)
print(h_awe)
print(np.size(AWE1_1))

產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472]
不同輸入資料的AWE是否相等: True
[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -20, 21, -21, 22, -22, 23, -23, 24, -24, 25, -25, 26, -26, 27, -27, 28, -28, 29, -29, 30, -30, 31, -31, 32, -32, 33, -33, 34, -34, 35, -35, 36, -36, 37, 

### 產生SEC的模數:

In [21]:
SEC_R = []
AWE1_R= []
for m in range(0,e1_1):
    mod = e1_data1[m] % AN
    SEC_R.append(mod)

for n in range(0,e1_1):
    mod_awe = AWE1_3[n] % AN
    AWE1_R.append(mod_awe)


print("SEC的模數:\n",SEC_R)
print("AWE1的模數:\n",SEC_R)
print("AWE1的模數是否等於SEC模數:", SEC_R == AWE1_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 13836, 2, 13835, 4, 13833, 8, 13829, 16, 13821, 32, 13805, 64, 13773, 128, 13709, 256, 13581, 512, 13325, 1024, 12813, 2048, 11789, 4096, 9741, 8192, 5645, 2547, 11290, 5094, 8743, 10188, 3649, 6539, 7298, 13078, 759, 12319, 1518, 10801, 3036, 7765, 6072, 1693, 12144, 3386, 10451, 6772, 7065, 13544, 293, 13251, 586, 12665, 1172, 11493, 2344, 9149, 4688, 4461, 9376, 8922, 4915, 4007, 9830, 8014, 5823, 2191, 11646, 4382, 9455, 8764, 5073, 3691, 10146]
AWE1的模數:
 [1, 13836, 2, 13835, 4, 13833, 8, 13829, 16, 13821, 32, 13805, 64, 13773, 128, 13709, 256, 13581, 512, 13325, 1024, 12813, 2048, 11789, 4096, 9741, 8192, 5645, 2547, 11290, 5094, 8743, 10188, 3649, 6539, 7298, 13078, 759, 12319, 1518, 10801, 3036, 7765, 6072, 1693, 12144, 3386, 10451, 6772, 7065, 13544, 293, 13251, 586, 12665, 1172, 11493, 2344, 9149, 4688, 4461, 9376, 8922, 4915, 4007, 9830, 8014, 5823, 2191, 11646, 4382, 9455, 8764, 5073, 3691, 10146]
AWE1的模數是否等於SEC模數: True
SEC的大小: 76


### 產生2-bit AWE (Arithmetic-Weight Error)

In [22]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
for i in range(0, bits1):
    for j in range(i+1, bits1):
        e2_data.insert(e2 , (input1 + 2**i + 2**j))
        AWE_2.insert(e2 , (2**i + 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 + 2**i - 2**j))
        AWE_2.insert(e2 , (2**i - 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i + 2**j))
        AWE_2.insert(e2 , (-2**i + 2**j))                
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i -2**j))
        AWE_2.insert(e2 , (-2**i - 2**j))
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        p = p + 1


print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits 錯誤資料:",e2_data)

第 0 筆錯誤資料: 232146047218
第 0 筆算術錯誤: 3
第 1 筆錯誤資料: 232146047214
第 1 筆算術錯誤: -1
第 2 筆錯誤資料: 232146047216
第 2 筆算術錯誤: 1
第 3 筆錯誤資料: 232146047212
第 3 筆算術錯誤: -3
第 4 筆錯誤資料: 232146047220
第 4 筆算術錯誤: 5
第 5 筆錯誤資料: 232146047212
第 5 筆算術錯誤: -3
第 6 筆錯誤資料: 232146047218
第 6 筆算術錯誤: 3
第 7 筆錯誤資料: 232146047210
第 7 筆算術錯誤: -5
第 8 筆錯誤資料: 232146047224
第 8 筆算術錯誤: 9
第 9 筆錯誤資料: 232146047208
第 9 筆算術錯誤: -7
第 10 筆錯誤資料: 232146047222
第 10 筆算術錯誤: 7
第 11 筆錯誤資料: 232146047206
第 11 筆算術錯誤: -9
第 12 筆錯誤資料: 232146047232
第 12 筆算術錯誤: 17
第 13 筆錯誤資料: 232146047200
第 13 筆算術錯誤: -15
第 14 筆錯誤資料: 232146047230
第 14 筆算術錯誤: 15
第 15 筆錯誤資料: 232146047198
第 15 筆算術錯誤: -17
第 16 筆錯誤資料: 232146047248
第 16 筆算術錯誤: 33
第 17 筆錯誤資料: 232146047184
第 17 筆算術錯誤: -31
第 18 筆錯誤資料: 232146047246
第 18 筆算術錯誤: 31
第 19 筆錯誤資料: 232146047182
第 19 筆算術錯誤: -33
第 20 筆錯誤資料: 232146047280
第 20 筆算術錯誤: 65
第 21 筆錯誤資料: 232146047152
第 21 筆算術錯誤: -63
第 22 筆錯誤資料: 232146047278
第 22 筆算術錯誤: 63
第 23 筆錯誤資料: 232146047150
第 23 筆算術錯誤: -65
第 24 筆錯誤資料: 232146047344
第 24 筆算術錯誤: 129
第 25 筆錯誤資料: 2321460

第 310 筆算術錯誤: 124
第 311 筆錯誤資料: 232146047083
第 311 筆算術錯誤: -132
第 312 筆錯誤資料: 232146047475
第 312 筆算術錯誤: 260
第 313 筆錯誤資料: 232146046963
第 313 筆算術錯誤: -252
第 314 筆錯誤資料: 232146047467
第 314 筆算術錯誤: 252
第 315 筆錯誤資料: 232146046955
第 315 筆算術錯誤: -260
第 316 筆錯誤資料: 232146047731
第 316 筆算術錯誤: 516
第 317 筆錯誤資料: 232146046707
第 317 筆算術錯誤: -508
第 318 筆錯誤資料: 232146047723
第 318 筆算術錯誤: 508
第 319 筆錯誤資料: 232146046699
第 319 筆算術錯誤: -516
第 320 筆錯誤資料: 232146048243
第 320 筆算術錯誤: 1028
第 321 筆錯誤資料: 232146046195
第 321 筆算術錯誤: -1020
第 322 筆錯誤資料: 232146048235
第 322 筆算術錯誤: 1020
第 323 筆錯誤資料: 232146046187
第 323 筆算術錯誤: -1028
第 324 筆錯誤資料: 232146049267
第 324 筆算術錯誤: 2052
第 325 筆錯誤資料: 232146045171
第 325 筆算術錯誤: -2044
第 326 筆錯誤資料: 232146049259
第 326 筆算術錯誤: 2044
第 327 筆錯誤資料: 232146045163
第 327 筆算術錯誤: -2052
第 328 筆錯誤資料: 232146051315
第 328 筆算術錯誤: 4100
第 329 筆錯誤資料: 232146043123
第 329 筆算術錯誤: -4092
第 330 筆錯誤資料: 232146051307
第 330 筆算術錯誤: 4092
第 331 筆錯誤資料: 232146043115
第 331 筆算術錯誤: -4100
第 332 筆錯誤資料: 232146055411
第 332 筆算術錯誤: 8196
第 333 筆錯誤資料: 

第 560 筆錯誤資料: 300865523959
第 560 筆算術錯誤: 68719476744
第 561 筆錯誤資料: 163426570487
第 561 筆算術錯誤: -68719476728
第 562 筆錯誤資料: 300865523943
第 562 筆算術錯誤: 68719476728
第 563 筆錯誤資料: 163426570471
第 563 筆算術錯誤: -68719476744
第 564 筆錯誤資料: 369585000695
第 564 筆算術錯誤: 137438953480
第 565 筆錯誤資料: 94707093751
第 565 筆算術錯誤: -137438953464
第 566 筆錯誤資料: 369585000679
第 566 筆算術錯誤: 137438953464
第 567 筆錯誤資料: 94707093735
第 567 筆算術錯誤: -137438953480
第 568 筆錯誤資料: 232146047263
第 568 筆算術錯誤: 48
第 569 筆錯誤資料: 232146047199
第 569 筆算術錯誤: -16
第 570 筆錯誤資料: 232146047231
第 570 筆算術錯誤: 16
第 571 筆錯誤資料: 232146047167
第 571 筆算術錯誤: -48
第 572 筆錯誤資料: 232146047295
第 572 筆算術錯誤: 80
第 573 筆錯誤資料: 232146047167
第 573 筆算術錯誤: -48
第 574 筆錯誤資料: 232146047263
第 574 筆算術錯誤: 48
第 575 筆錯誤資料: 232146047135
第 575 筆算術錯誤: -80
第 576 筆錯誤資料: 232146047359
第 576 筆算術錯誤: 144
第 577 筆錯誤資料: 232146047103
第 577 筆算術錯誤: -112
第 578 筆錯誤資料: 232146047327
第 578 筆算術錯誤: 112
第 579 筆錯誤資料: 232146047071
第 579 筆算術錯誤: -144
第 580 筆錯誤資料: 232146047487
第 580 筆算術錯誤: 272
第 581 筆錯誤資料: 232146046975
第 5

第 810 筆錯誤資料: 240735981775
第 810 筆算術錯誤: 8589934560
第 811 筆錯誤資料: 223556112591
第 811 筆算術錯誤: -8589934624
第 812 筆錯誤資料: 249325916431
第 812 筆算術錯誤: 17179869216
第 813 筆錯誤資料: 214966178063
第 813 筆算術錯誤: -17179869152
第 814 筆錯誤資料: 249325916367
第 814 筆算術錯誤: 17179869152
第 815 筆錯誤資料: 214966177999
第 815 筆算術錯誤: -17179869216
第 816 筆錯誤資料: 266505785615
第 816 筆算術錯誤: 34359738400
第 817 筆錯誤資料: 197786308879
第 817 筆算術錯誤: -34359738336
第 818 筆錯誤資料: 266505785551
第 818 筆算術錯誤: 34359738336
第 819 筆錯誤資料: 197786308815
第 819 筆算術錯誤: -34359738400
第 820 筆錯誤資料: 300865523983
第 820 筆算術錯誤: 68719476768
第 821 筆錯誤資料: 163426570511
第 821 筆算術錯誤: -68719476704
第 822 筆錯誤資料: 300865523919
第 822 筆算術錯誤: 68719476704
第 823 筆錯誤資料: 163426570447
第 823 筆算術錯誤: -68719476768
第 824 筆錯誤資料: 369585000719
第 824 筆算術錯誤: 137438953504
第 825 筆錯誤資料: 94707093775
第 825 筆算術錯誤: -137438953440
第 826 筆錯誤資料: 369585000655
第 826 筆算術錯誤: 137438953440
第 827 筆錯誤資料: 94707093711
第 827 筆算術錯誤: -137438953504
第 828 筆錯誤資料: 232146047407
第 828 筆算術錯誤: 192
第 829 筆錯誤資料: 232146047151
第 82

第 1059 筆算術錯誤: -17179869312
第 1060 筆錯誤資料: 266505785711
第 1060 筆算術錯誤: 34359738496
第 1061 筆錯誤資料: 197786308975
第 1061 筆算術錯誤: -34359738240
第 1062 筆錯誤資料: 266505785455
第 1062 筆算術錯誤: 34359738240
第 1063 筆錯誤資料: 197786308719
第 1063 筆算術錯誤: -34359738496
第 1064 筆錯誤資料: 300865524079
第 1064 筆算術錯誤: 68719476864
第 1065 筆錯誤資料: 163426570607
第 1065 筆算術錯誤: -68719476608
第 1066 筆錯誤資料: 300865523823
第 1066 筆算術錯誤: 68719476608
第 1067 筆錯誤資料: 163426570351
第 1067 筆算術錯誤: -68719476864
第 1068 筆錯誤資料: 369585000815
第 1068 筆算術錯誤: 137438953600
第 1069 筆錯誤資料: 94707093871
第 1069 筆算術錯誤: -137438953344
第 1070 筆錯誤資料: 369585000559
第 1070 筆算術錯誤: 137438953344
第 1071 筆錯誤資料: 94707093615
第 1071 筆算術錯誤: -137438953600
第 1072 筆錯誤資料: 232146047983
第 1072 筆算術錯誤: 768
第 1073 筆錯誤資料: 232146046959
第 1073 筆算術錯誤: -256
第 1074 筆錯誤資料: 232146047471
第 1074 筆算術錯誤: 256
第 1075 筆錯誤資料: 232146046447
第 1075 筆算術錯誤: -768
第 1076 筆錯誤資料: 232146048495
第 1076 筆算術錯誤: 1280
第 1077 筆錯誤資料: 232146046447
第 1077 筆算術錯誤: -768
第 1078 筆錯誤資料: 232146047983
第 1078 筆算術錯誤: 768
第 1079 筆錯誤

...

第 1809 筆錯誤資料: 232145817839
第 1809 筆算術錯誤: -229376
第 1810 筆錯誤資料: 232146276591
第 1810 筆算術錯誤: 229376
第 1811 筆錯誤資料: 232145752303
第 1811 筆算術錯誤: -294912
第 1812 筆錯誤資料: 232146604271
第 1812 筆算術錯誤: 557056
第 1813 筆錯誤資料: 232145555695
第 1813 筆算術錯誤: -491520
第 1814 筆錯誤資料: 232146538735
第 1814 筆算術錯誤: 491520
第 1815 筆錯誤資料: 232145490159
第 1815 筆算術錯誤: -557056
第 1816 筆錯誤資料: 232147128559
第 1816 筆算術錯誤: 1081344
第 1817 筆錯誤資料: 232145031407
第 1817 筆算術錯誤: -1015808
第 1818 筆錯誤資料: 232147063023
第 1818 筆算術錯誤: 1015808
第 1819 筆錯誤資料: 232144965871
第 1819 筆算術錯誤: -1081344
第 1820 筆錯誤資料: 232148177135
第 1820 筆算術錯誤: 2129920
第 1821 筆錯誤資料: 232143982831
第 1821 筆算術錯誤: -2064384
第 1822 筆錯誤資料: 232148111599
第 1822 筆算術錯誤: 2064384
第 1823 筆錯誤資料: 232143917295
第 1823 筆算術錯誤: -2129920
第 1824 筆錯誤資料: 232150274287
第 1824 筆算術錯誤: 4227072
第 1825 筆錯誤資料: 232141885679
第 1825 筆算術錯誤: -4161536
第 1826 筆錯誤資料: 232150208751
第 1826 筆算術錯誤: 4161536
第 1827 筆錯誤資料: 232141820143
第 1827 筆算術錯誤: -4227072
第 1828 筆錯誤資料: 232154468591
第 1828 筆算術錯誤: 8421376
第 1829 筆錯誤資料: 232

第 2058 筆算術錯誤: 786432
第 2059 筆錯誤資料: 232144736495
第 2059 筆算術錯誤: -1310720
第 2060 筆錯誤資料: 232148406511
第 2060 筆算術錯誤: 2359296
第 2061 筆錯誤資料: 232144212207
第 2061 筆算術錯誤: -1835008
第 2062 筆錯誤資料: 232147882223
第 2062 筆算術錯誤: 1835008
第 2063 筆錯誤資料: 232143687919
第 2063 筆算術錯誤: -2359296
第 2064 筆錯誤資料: 232150503663
第 2064 筆算術錯誤: 4456448
第 2065 筆錯誤資料: 232142115055
第 2065 筆算術錯誤: -3932160
第 2066 筆錯誤資料: 232149979375
第 2066 筆算術錯誤: 3932160
第 2067 筆錯誤資料: 232141590767
第 2067 筆算術錯誤: -4456448
第 2068 筆錯誤資料: 232154697967
第 2068 筆算術錯誤: 8650752
第 2069 筆錯誤資料: 232137920751
第 2069 筆算術錯誤: -8126464
第 2070 筆錯誤資料: 232154173679
第 2070 筆算術錯誤: 8126464
第 2071 筆錯誤資料: 232137396463
第 2071 筆算術錯誤: -8650752
第 2072 筆錯誤資料: 232163086575
第 2072 筆算術錯誤: 17039360
第 2073 筆錯誤資料: 232129532143
第 2073 筆算術錯誤: -16515072
第 2074 筆錯誤資料: 232162562287
第 2074 筆算術錯誤: 16515072
第 2075 筆錯誤資料: 232129007855
第 2075 筆算術錯誤: -17039360
第 2076 筆錯誤資料: 232179863791
第 2076 筆算術錯誤: 33816576
第 2077 筆錯誤資料: 232112754927
第 2077 筆算術錯誤: -33292288
第 2078 筆錯誤資料: 232179339503
第 207

第 2308 筆算術錯誤: 4297064448
第 2309 筆錯誤資料: 227853177071
第 2309 筆算術錯誤: -4292870144
第 2310 筆錯誤資料: 236438917359
第 2310 筆算術錯誤: 4292870144
第 2311 筆錯誤資料: 227848982767
第 2311 筆算術錯誤: -4297064448
第 2312 筆錯誤資料: 240738078959
第 2312 筆算術錯誤: 8592031744
第 2313 筆錯誤資料: 223558209775
第 2313 筆算術錯誤: -8587837440
第 2314 筆錯誤資料: 240733884655
第 2314 筆算術錯誤: 8587837440
第 2315 筆錯誤資料: 223554015471
第 2315 筆算術錯誤: -8592031744
第 2316 筆錯誤資料: 249328013551
第 2316 筆算術錯誤: 17181966336
第 2317 筆錯誤資料: 214968275183
第 2317 筆算術錯誤: -17177772032
第 2318 筆錯誤資料: 249323819247
第 2318 筆算術錯誤: 17177772032
第 2319 筆錯誤資料: 214964080879
第 2319 筆算術錯誤: -17181966336
第 2320 筆錯誤資料: 266507882735
第 2320 筆算術錯誤: 34361835520
第 2321 筆錯誤資料: 197788405999
第 2321 筆算術錯誤: -34357641216
第 2322 筆錯誤資料: 266503688431
第 2322 筆算術錯誤: 34357641216
第 2323 筆錯誤資料: 197784211695
第 2323 筆算術錯誤: -34361835520
第 2324 筆錯誤資料: 300867621103
第 2324 筆算術錯誤: 68721573888
第 2325 筆錯誤資料: 163428667631
第 2325 筆算術錯誤: -68717379584
第 2326 筆錯誤資料: 300863426799
第 2326 筆算術錯誤: 68717379584
第 2327 筆錯誤資料: 16342

第 2558 筆錯誤資料: 232615809263
第 2558 筆算術錯誤: 469762048
第 2559 筆錯誤資料: 231542067439
第 2559 筆算術錯誤: -603979776
第 2560 筆錯誤資料: 233286897903
第 2560 筆算術錯誤: 1140850688
第 2561 筆錯誤資料: 231139414255
第 2561 筆算術錯誤: -1006632960
第 2562 筆錯誤資料: 233152680175
第 2562 筆算術錯誤: 1006632960
第 2563 筆錯誤資料: 231005196527
第 2563 筆算術錯誤: -1140850688
第 2564 筆錯誤資料: 234360639727
第 2564 筆算術錯誤: 2214592512
第 2565 筆錯誤資料: 230065672431
第 2565 筆算術錯誤: -2080374784
第 2566 筆錯誤資料: 234226421999
第 2566 筆算術錯誤: 2080374784
第 2567 筆錯誤資料: 229931454703
第 2567 筆算術錯誤: -2214592512
第 2568 筆錯誤資料: 236508123375
第 2568 筆算術錯誤: 4362076160
第 2569 筆錯誤資料: 227918188783
第 2569 筆算術錯誤: -4227858432
第 2570 筆錯誤資料: 236373905647
第 2570 筆算術錯誤: 4227858432
第 2571 筆錯誤資料: 227783971055
第 2571 筆算術錯誤: -4362076160
第 2572 筆錯誤資料: 240803090671
第 2572 筆算術錯誤: 8657043456
第 2573 筆錯誤資料: 223623221487
第 2573 筆算術錯誤: -8522825728
第 2574 筆錯誤資料: 240668872943
第 2574 筆算術錯誤: 8522825728
第 2575 筆錯誤資料: 223489003759
第 2575 筆算術錯誤: -8657043456
第 2576 筆錯誤資料: 249393025263
第 2576 筆算術錯誤: 17246978048
第 25

第 2808 筆錯誤資料: 438304477423
第 2808 筆算術錯誤: 206158430208
第 2809 筆錯誤資料: 163426570479
第 2809 筆算術錯誤: -68719476736
第 2810 筆錯誤資料: 300865523951
第 2810 筆算術錯誤: 68719476736
第 2811 筆錯誤資料: 25987617007
第 2811 筆算術錯誤: -206158430208
2812
703
產生2-bits AWE: [3, -1, 1, -3, 5, -3, 3, -5, 9, -7, 7, -9, 17, -15, 15, -17, 33, -31, 31, -33, 65, -63, 63, -65, 129, -127, 127, -129, 257, -255, 255, -257, 513, -511, 511, -513, 1025, -1023, 1023, -1025, 2049, -2047, 2047, -2049, 4097, -4095, 4095, -4097, 8193, -8191, 8191, -8193, 16385, -16383, 16383, -16385, 32769, -32767, 32767, -32769, 65537, -65535, 65535, -65537, 131073, -131071, 131071, -131073, 262145, -262143, 262143, -262145, 524289, -524287, 524287, -524289, 1048577, -1048575, 1048575, -1048577, 2097153, -2097151, 2097151, -2097153, 4194305, -4194303, 4194303, -4194305, 8388609, -8388607, 8388607, -8388609, 16777217, -16777215, 16777215, -16777217, 33554433, -33554431, 33554431, -33554433, 67108865, -67108863, 67108863, -67108865, 134217729, -134217727, 13

In [23]:
DEC_R = []
for m in range(0,e2):
    mod = e2_data[m] % AN
    DEC_R.append(mod)

print("DEC_R的模數:\n",DEC_R)
print(np.size(DEC_R))

DEC_R的模數:
 [3, 13836, 1, 13834, 5, 13834, 3, 13832, 9, 13830, 7, 13828, 17, 13822, 15, 13820, 33, 13806, 31, 13804, 65, 13774, 63, 13772, 129, 13710, 127, 13708, 257, 13582, 255, 13580, 513, 13326, 511, 13324, 1025, 12814, 1023, 12812, 2049, 11790, 2047, 11788, 4097, 9742, 4095, 9740, 8193, 5646, 8191, 5644, 2548, 11291, 2546, 11289, 5095, 8744, 5093, 8742, 10189, 3650, 10187, 3648, 6540, 7299, 6538, 7297, 13079, 760, 13077, 758, 12320, 1519, 12318, 1517, 10802, 3037, 10800, 3035, 7766, 6073, 7764, 6071, 1694, 12145, 1692, 12143, 3387, 10452, 3385, 10450, 6773, 7066, 6771, 7064, 13545, 294, 13543, 292, 13252, 587, 13250, 585, 12666, 1173, 12664, 1171, 11494, 2345, 11492, 2343, 9150, 4689, 9148, 4687, 4462, 9377, 4460, 9375, 8923, 4916, 8921, 4914, 4008, 9831, 4006, 9829, 8015, 5824, 8013, 5822, 2192, 11647, 2190, 11645, 4383, 9456, 4381, 9454, 8765, 5074, 8763, 5072, 3692, 10147, 3690, 10145, 6, 13835, 2, 13831, 10, 13831, 6, 13827, 18, 13823, 14, 13819, 34, 13807, 30, 13803, 66, 13775

In [24]:
total_AWE = AWE1_1 + AWE_2
e_total = e1_data1 + e2_data
R_total = SEC_R + DEC_R
print("還未去除重複error data的數量:",np.size(e_total))
print("還未去除重複餘數的數量:", np.size(R_total))

## 把重複的 AWE 及 error_data 去除
AWE = remove_duplicates(total_AWE)
new_e_total = remove_duplicates(e_total)
new_R_total = remove_duplicates(R_total)
error_total_count = np.size(new_e_total)
print("已去除重複error data的數量:",error_total_count)
print("已未去除重複餘數的數量:",np.size(new_R_total))
print("已未去除重複AWE的數量:",np.size(AWE))
print("所有的AWE為:\n",AWE)
print("所有算術錯誤的資料:\n", new_e_total)

還未去除重複error data的數量: 2888
還未去除重複餘數的數量: 2888
已去除重複error data的數量: 2742
已未去除重複餘數的數量: 2742
已未去除重複AWE的數量: 2742
所有的AWE為:
 [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472, 3, -3, 5, -5, 9, -7, 7, -9, 17, -15, 15, -17, 33, -31, 31, -33, 65, -63, 63, -65, 129, -127, 127, -129, 257, -255, 255, -257, 513, -511, 511, -513, 1025, -1023, 1023, -1025, 2049, -2047, 2047, -2049, 4097, -4095, 4095, -4097, 819

### 所有AWE數量: $4*C^{bits}_{2}$ $-$ $2*(bits-1)$ - $2*(bits-2)$
* $2*(bits-1)$: 所有的 single AWE都可以分解成已重複的 double AWE, 除了MSB例外
* EX: $(+,0)$ 只能分解成 $(0,++)$ , $(-,0)$ 只能分解成 $(0,--)$
* $2*(bits-2)$: double AWE重複的個數
* EX: $(+,0,-)$ = $(0,+,+)$ and $(-,0,+)$ = $(0,-,-)$

### Trade-off Algorithm: 2025/05/27 使用 Vivado HLX IP 驗證
* 此解出來的數字為位置及正負號 EX: $-1$ 代表 $-2^0$ , $-5$ 代表 $-2^4$ 以此類推

In [25]:
import math
import time
### 給定一個DEC_R, 找錯誤位置h1及h2(包含錯誤正負號)
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]

start_time = time.time()  # 記錄開始時間
for R, error in zip(new_R_total, new_e_total): 
    ip.write_mm(0, R)
    
    h1 = twos_complement_32bit(ip.read_mm(4))                  
    h2 = twos_complement_32bit(ip.read_mm(8))
    
    s2 = get_sign(h2)
    s1 = get_sign(h1)
    
    possible_h1.append(h1)                  
    possible_h2.append(h2)                  
    output.append((error - (s1 * 2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // AN)

            
end_time = time.time()  # 記錄結束時間
elapsed_time = end_time - start_time  # 計算執行時間
print(f"執行時間: {elapsed_time:.6f} 秒")
print("更正後的值是否和原data相同:", data1 == output.pop())
print(elapsed_time / np.size(possible_h1))
## print(output)
### print(possible_h1)
### print(possible_h2)
##  print("更正後的值:",output)
print(np.size(possible_h1))
print(np.size(possible_h2))
##print("輸入的 error data的總數是否和經過演算法後相同:", error_total_count == np.size(possible_h1))
##print(possible_SE_h1)

執行時間: 0.308407 秒
更正後的值是否和原data相同: True
0.00011247505157381665
2742
2742
